# Experimental notebook for models

## Import libs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys,inspect
#sys.path.insert(0,"..")
os.chdir('..')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2 

from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable

import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch

from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset

# Reproducibility
random.seed(0)
os.environ['PYTHONHASHSEED'] = str(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# Device
DEVICE = "cpu" #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# EXPERIMENT_NAME = "coin_unet_isic2018"

# ROOT_DIR = os.path.abspath(".")
# LOG_PATH = os.path.join(ROOT_DIR, "logs", EXPERIMENT_NAME)

# if not os.path.exists(os.path.join(ROOT_DIR, "logs")):
#     os.mkdir(os.path.join(ROOT_DIR, "logs"))
    
# if not os.path.exists(LOG_PATH):
#     os.mkdir(LOG_PATH)

In [ ]:
!pwd

In [ ]:
from models.unet import build_unet
from models.LeViTUNet128s import Build_LeViT_UNet_128s
from models.LeViTUNet192 import Build_LeViT_UNet_192
from models.LeViTUNet384 import Build_LeViT_UNet_384
from models.unetplusplus import NestedUNet
from models.resnet import rf_lw50, rf_lw101, rf_lw152

In [ ]:
import torch
from segformer_pytorch import Segformer

model = Segformer(
    dims = (32, 64, 160, 256),      # dimensions of each stage
    heads = (1, 2, 5, 8),           # heads of each stage
    ff_expansion = (8, 8, 4, 4),    # feedforward expansion factor of each stage
    reduction_ratio = (8, 4, 2, 1), # reduction ratio of each stage for efficient attention
    num_layers = 2,                 # num layers of each stage
    decoder_dim = 256,              # decoder dimension
    num_classes = 40                 # number of segmentation classes
)

model

In [ ]:
x = torch.randn(1, 3, 224, 224)
pred = model(x) # (1, 4, 64, 64)  # output is (H/4, W/4) map of the number of segmentation classes

In [ ]:
pred.shape

In [ ]:
model = rf_lw152(40, imagenet=True)
model

In [ ]:
model.eval()

output = model(torch.randn(1, 3, 224, 224))
output.shape

In [ ]:
target = torch.randn(1, 224, 224)

In [ ]:
output = (
    cv2.resize(
        output[0, :40].data.cpu().numpy().transpose(1, 2, 0),
        target.size()[1:][::-1],
        interpolation=cv2.INTER_CUBIC,
    )
    .argmax(axis=2)
    .astype(np.uint8)
            )

In [ ]:
output.shape

In [ ]:
output

In [ ]:
x = F.interpolate(output, size=(224, 224), mode="bilinear", align_corners=True)
x.shape

In [ ]:
# LeViT_128S 304340946 FLOPs 15894627 parameters
# the number of parameters: 15894627 ==> 15.89 M
# Computational complexity:       17.42 GMac
# Number of parameters:           15.89 M 

# LeViT_192 656833916 FLOPs 19894150 parameters
# the number of parameters: 19894150 ==> 19.89 M

# Computational complexity:       25.4 GMac
# Number of parameters:           52.15 M 
# LeViT_384 2351399380 FLOPs 52154213 parameters
# the number of parameters: 52154213 ==> 52.15 M

In [ ]:
model = NestedUNet()

model.eval()

output = model(torch.randn(1, 3, 224, 224))
output.shape

In [ ]:
model = ODOC_seg_edge()

model.eval()

output = model(torch.randn(1, 3, 256, 256))
output.shape

In [ ]:
model = Build_LeViT_UNet_128s(num_classes=1, pretrained=True)

model.eval()

output = model(torch.randn(1, 3, 224, 224))
output.shape

In [ ]:
# From train script

# 128s
# All parameters  15894627
# Trainable parameters  15894627

